<a href="https://colab.research.google.com/github/JoshitaReddy/Text-Summariser/blob/main/Text_Summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') 
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-11-08 15:48:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-08 15:48:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-08 15:48:43--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Reading Text

In [ ]:
f = open("text.txt", "r")
text=f.read()
print(text)
f.close()

We have already done great damage to Mother Earth. Now nations across the world have understood the urgency of the global warming issue, and are taking action to prevent further rising of global temperatures.

There have been many agreements on ways to reduce the emission of greenhouse gases in the past.
The latest one is the Paris agreement by the United Nations Framework Convention on Climate Change (UNFCCC) members.
The main goal of this agreement is to keep the increase in global average temperature to well below 2 °C above pre-industrial levels and to limit the increase to 1.5 °C.
To offset carbon emissions, the Indian government is taking various initiatives. These include the solar mission, electrification of transport, clean development mechanisms, effective utilization of natural resources such as wind energy, hydel power, etc.

Our government has introduced schemes to reduce the usage of unclean fuels for cooking in rural households.
As individuals, we too have the responsibi

Text To Sentences

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text)

Data Preprocessing

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

Vector Representation of Sentences

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in sentences_clean:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

Similarity Matrix

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

PageRank Algorithm

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

Extract Summary

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
for i in range(5):
  print(ranked_sentences[i][1])

Recycling has to become a habit in our society as the production of new components takes lots of energy and leads to pollution too.
There have been many agreements on ways to reduce the emission of greenhouse gases in the past.
Now nations across the world have understood the urgency of the global warming issue, and are taking action to prevent further rising of global temperatures.
We can drive less and use public transport instead.
As individuals, we too have the responsibility in making our planet a safe abode for our future generations.
